In [40]:
import pandas as pd
import math
import numpy
df_ref_transcript_data = pd.read_csv('/Users/pankajverma/Desktop/summerInternship2k17/GSoC/Ensembl/1.lncRNA/dataset/refseq_numeric/all_type=lncRNA_data/all_lncrna_sorted_chr/all_lncrna_transcript_data_sequence_sorted_chr.csv', index_col=0)

In [41]:
df_ref_transcript_data.head(3)

,SNO,Index,RefSeq Gene ID,RefSeq Transcript ID,Dbxref,Parent,gbkey,Type,Seq_id,Strand,Start,End,No. of Exons,Transcript Length,Length of Sequences,| Seq_len - Trans_len |,Match ?,Sequences
0,1,27,gene-MIR1302-2HG,rna-XR_001737835.1,"GeneID:107985730,Genbank:XR_001737835.1,HGNC:H...",gene-MIR1302-2HG,ncRNA,lnc_RNA,1,1,29926,31295,3,538,538.0,0.0,YES,TCGTATGAGTCCAGTTTTGTGAAGTGCCTAGAATAGTCAAAATTCA...
1,1508,251564,gene-LOC107985451,rna-XR_001737807.1,"GeneID:107985451,Genbank:XR_001737807.1",gene-LOC107985451,ncRNA,lnc_RNA,1,-1,164909668,164912572,3,603,603.0,0.0,YES,GCCATTAAGGGATGAGCGTAAGCAGTGACTGACAGGCAGGCTCTGC...
2,1507,251556,gene-LOC100505795,rna-NR_038072.1,"GeneID:100505795,Genbank:NR_038072.1",gene-LOC100505795,ncRNA,lnc_RNA,1,-1,164769116,164774641,4,1963,1963.0,0.0,YES,CTTATTAAAAATAGAATATTGCCTACATCACATAAAGGGCAAACTC...


In [42]:
ref_all_seq = df_ref_transcript_data['Sequences'].tolist()

In [43]:
# ens_all_seq[0]

In [44]:
len(df_ref_transcript_data)

25715

In [45]:
from Bio import Seq
import regex as re
from tqdm import tqdm

In [74]:
no_of_amino_acids = []
len_longest_orf = []
longest_orf_seq = []
start_seq_index = []
protein_seq_encoded = []

In [75]:
# no_of_amino_acids

In [76]:
%%time
def get_all_orf_info(input_seq):
    startP = re.compile('ATG')
#     nuc = input_seq.replace('\n','')
    nuc = input_seq
    longest = (0,)
    for m in startP.finditer(nuc, overlapped=True):
        if len(Seq.Seq(nuc)[m.start():].translate(to_stop=True)) > longest[0]:
            pro = Seq.Seq(nuc)[m.start():].translate(to_stop=True)
            longest = (len(pro), 
                       m.start(), 
                       str(pro),
                       nuc[m.start():m.start()+len(pro)*3+3])
            return(longest)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs


In [77]:
for i in tqdm(range(len(ref_all_seq))):
    
    if type(ref_all_seq[i]) is float:
        no_of_amino_acids.append('Nan')
        len_longest_orf.append('Nan')
        start_seq_index.append('Nan')
        protein_seq_encoded.append('Nan')
        longest_orf_seq.append('Nan')
        continue
    
    else:
        
        longest = get_all_orf_info(ref_all_seq[i])
        if longest == None or longest[0] == 0:
            no_of_amino_acids.append('No start codon found')
            len_longest_orf.append('No start codon found')
            start_seq_index.append('No start codon found')
            protein_seq_encoded.append('No start codon found')
            longest_orf_seq.append('No start codon found')
            continue

        else:

            no_of_amino_acids.append(longest[0])
            len_longest_orf.append((len(longest[3])-3))
            start_seq_index.append(longest[1])
            protein_seq_encoded.append(longest[2])
            longest_orf_seq.append(longest[3])




  0%|          | 0/25715 [00:00<?, ?it/s]


  1%|▏         | 328/25715 [00:00<00:07, 3194.51it/s]


  2%|▏         | 473/25715 [00:00<00:10, 2346.92it/s]


  3%|▎         | 691/25715 [00:00<00:10, 2293.44it/s]


  3%|▎         | 834/25715 [00:00<00:12, 1932.18it/s]


  4%|▍         | 977/25715 [00:00<00:15, 1609.97it/s]


  4%|▍         | 1112/25715 [00:00<00:17, 1415.18it/s]


  5%|▍         | 1240/25715 [00:00<00:19, 1270.85it/s]


  5%|▌         | 1360/25715 [00:00<00:20, 1197.91it/s]


  6%|▌         | 1476/25715 [00:01<00:24, 1006.96it/s]


  6%|▌         | 1584/25715 [00:01<00:23, 1027.46it/s]


  8%|▊         | 1944/25715 [00:01<00:18, 1307.67it/s]


  9%|▉         | 2306/25715 [00:01<00:14, 1617.66it/s]


 10%|▉         | 2542/25715 [00:01<00:13, 1702.71it/s]


 11%|█         | 2765/25715 [00:01<00:14, 1546.83it/s]


 12%|█▏        | 2960/25715 [00:01<00:17, 1331.48it/s]


 12%|█▏        | 3127/25715 [00:02<00:18, 1213.98it/s]


 13%|█▎        | 3369/25715 [00:02<00:15, 1427

In [88]:
stop_codon = []

In [89]:
longest_orf_seq[0][-3:]

'TGA'

In [90]:
for i in tqdm(range(len(longest_orf_seq))):
    if longest_orf_seq[i] == 'No start codon found':
        stop_codon.append('No start codon found')
        continue
        
    if longest_orf_seq[i] == 'Nan':
        stop_codon.append('Nan')
        continue
        
    else:
        if longest_orf_seq[i][-3:] == 'TAA' or longest_orf_seq[i][-3:] == 'TAG' or longest_orf_seq[i][-3:] == 'TGA':
            stop_codon.append('YES')
        else:
            stop_codon.append('NO')




  0%|          | 0/25715 [00:00<?, ?it/s]


100%|██████████| 25715/25715 [00:00<00:00, 367688.11it/s]

In [91]:
len(stop_codon)

25715

In [92]:
c = 0
index = []
for i in range(len(stop_codon)):
    if stop_codon[i] == 'NO':
        c = c + 1
        index.append(i)
        

In [93]:
c

111

In [94]:
orf_multiple_of_3 = []

In [95]:
# longest_orf_seq

In [96]:
c = 0
index = []
for i in range(len(stop_codon)):
    if len_longest_orf[i] == 'No start codon found':
        orf_multiple_of_3.append('No start codon found')
        continue
        
    if len_longest_orf[i] == 'Nan':
        orf_multiple_of_3.append('Nan')
        continue
        
    else:
        if len_longest_orf[i] % 3 == 0:
            orf_multiple_of_3.append('YES')
        else:
            orf_multiple_of_3.append('NO')
            c = c + 1
            index.append(i)
        

In [97]:
c

73

In [98]:
# no_of_amino_acids = []
# len_longest_orf = []
# longest_orf_seq = []
# start_seq_index = []
# protein_seq_encoded = []
# orf_multiple_of_3 = []
# stop_codon = []

In [99]:
# orf_multiple_of_3

In [100]:
df_ref_transcript_data['Len of longest ORF'] = len_longest_orf
df_ref_transcript_data['No. of Amino Acids'] = no_of_amino_acids
df_ref_transcript_data['Longest ORF Seq'] = longest_orf_seq
df_ref_transcript_data['ORF Start Index for longest Seq'] = start_seq_index
df_ref_transcript_data['Protein Seq encoded'] = protein_seq_encoded
df_ref_transcript_data['Stop codon present ?'] = stop_codon
df_ref_transcript_data['ORF multiple of 3 ?'] = orf_multiple_of_3

In [101]:
df_ref_transcript_data.head(3)

,SNO,Index,RefSeq Gene ID,RefSeq Transcript ID,Dbxref,Parent,gbkey,Type,Seq_id,Strand,...,| Seq_len - Trans_len |,Match ?,Sequences,Len of longest ORF,No. of Amino Acids,Longest ORF Seq,ORF Start Index for longest Seq,Protein Seq encoded,Stop codon present ?,ORF multiple of 3 ?
0,1,27,gene-MIR1302-2HG,rna-XR_001737835.1,"GeneID:107985730,Genbank:XR_001737835.1,HGNC:H...",gene-MIR1302-2HG,ncRNA,lnc_RNA,1,1,...,0.0,YES,TCGTATGAGTCCAGTTTTGTGAAGTGCCTAGAATAGTCAAAATTCA...,15,5,ATGAGTCCAGTTTTGTGA,4,MSPVL,YES,YES
1,1508,251564,gene-LOC107985451,rna-XR_001737807.1,"GeneID:107985451,Genbank:XR_001737807.1",gene-LOC107985451,ncRNA,lnc_RNA,1,-1,...,0.0,YES,GCCATTAAGGGATGAGCGTAAGCAGTGACTGACAGGCAGGCTCTGC...,18,6,ATGAGCGTAAGCAGTGACTGA,11,MSVSSD,YES,YES
2,1507,251556,gene-LOC100505795,rna-NR_038072.1,"GeneID:100505795,Genbank:NR_038072.1",gene-LOC100505795,ncRNA,lnc_RNA,1,-1,...,0.0,YES,CTTATTAAAAATAGAATATTGCCTACATCACATAAAGGGCAAACTC...,123,41,ATGGGCACACTTGGTGATGTGCAACTCAGTGGGGCCTACAGGGTCC...,118,MGTLGDVQLSGAYRVPADIDITKEFEPGLRNRVSGCFKKAV,YES,YES


In [102]:
df_ref_transcript_data.iloc[304]

SNO                                                                             1202
Index                                                                         192219
RefSeq Gene ID                                                     gene-LOC100996263
RefSeq Transcript ID                                                 rna-NR_121626.1
Dbxref                                          GeneID:100996263,Genbank:NR_121626.1
Parent                                                             gene-LOC100996263
gbkey                                                                          ncRNA
Type                                                                         lnc_RNA
Seq_id                                                                             1
Strand                                                                            -1
Start                                                                      117596832
End                                                              

In [103]:
df_ref_transcript_data.to_csv('/Users/pankajverma/Desktop/summerInternship2k17/GSoC/Ensembl/1.lncRNA/feature_selection/ORF/RefSeq/all_lncrna_transcript_data_orf_info.csv')

In [104]:
df_ref_transcript_data.iloc[index[0]]

SNO                                                                             1997
Index                                                                         313776
RefSeq Gene ID                                                     gene-LOC105372984
RefSeq Transcript ID                                              rna-XR_001738488.1
Dbxref                                       GeneID:105372984,Genbank:XR_001738488.1
Parent                                                             gene-LOC105372984
gbkey                                                                          ncRNA
Type                                                                         lnc_RNA
Seq_id                                                                             1
Strand                                                                            -1
Start                                                                      222751264
End                                                              

In [1]:
import pandas as pd
df_ref_transcript_data = pd.read_csv('/Users/pankajverma/Desktop/summerInternship2k17/GSoC/Ensembl/1.lncRNA/feature_selection/ORF/RefSeq/all_lncrna_transcript_data_orf_info.csv', index_col=0)

In [2]:
df_ref_transcript_data.head(3)

,SNO,Index,RefSeq Gene ID,RefSeq Transcript ID,Dbxref,Parent,gbkey,Type,Seq_id,Strand,...,Match ?,Sequences,GC %,Len of longest ORF,No. of Amino Acids,Longest ORF Seq,ORF Start Index for longest Seq,Protein Seq encoded,Stop codon present ?,ORF multiple of 3 ?
0,1,27,gene-MIR1302-2HG,rna-XR_001737835.1,"GeneID:107985730,Genbank:XR_001737835.1,HGNC:H...",gene-MIR1302-2HG,ncRNA,lnc_RNA,1,1,...,YES,TCGTATGAGTCCAGTTTTGTGAAGTGCCTAGAATAGTCAAAATTCA...,51.3,15,5,ATGAGTCCAGTTTTGTGA,4,MSPVL,YES,YES
1,1508,251564,gene-LOC107985451,rna-XR_001737807.1,"GeneID:107985451,Genbank:XR_001737807.1",gene-LOC107985451,ncRNA,lnc_RNA,1,-1,...,YES,GCCATTAAGGGATGAGCGTAAGCAGTGACTGACAGGCAGGCTCTGC...,51.74,18,6,ATGAGCGTAAGCAGTGACTGA,11,MSVSSD,YES,YES
2,1507,251556,gene-LOC100505795,rna-NR_038072.1,"GeneID:100505795,Genbank:NR_038072.1",gene-LOC100505795,ncRNA,lnc_RNA,1,-1,...,YES,CTTATTAAAAATAGAATATTGCCTACATCACATAAAGGGCAAACTC...,44.27,123,41,ATGGGCACACTTGGTGATGTGCAACTCAGTGGGGCCTACAGGGTCC...,118,MGTLGDVQLSGAYRVPADIDITKEFEPGLRNRVSGCFKKAV,YES,YES


In [3]:
from tqdm import tqdm

c = 0
index = []
for i in tqdm(range(len(df_ref_transcript_data))):
    if df_ref_transcript_data['Stop codon present ?'][i] == 'YES' and df_ref_transcript_data['ORF multiple of 3 ?'][i] == 'NO':
        c = c + 1
        index.append(i)

100%|██████████| 25715/25715 [00:00<00:00, 27012.72it/s]


In [4]:
c

1

In [36]:
index

[4048]

In [37]:
df_ref_transcript_data.iloc[4048]

SNO                                                                             3297
Index                                                                         485215
RefSeq Gene ID                                                           gene-GACAT1
RefSeq Transcript ID                                                 rna-NR_126369.1
Dbxref                             GeneID:104326057,Genbank:NR_126369.1,HGNC:HGNC...
Parent                                                                   gene-GACAT1
gbkey                                                                          ncRNA
Type                                                                         lnc_RNA
Seq_id                                                                             2
Strand                                                                             1
Start                                                                      107754112
End                                                              

In [38]:
from Bio.SeqUtils import GC123
from tqdm import tqdm

In [39]:
gc_percentage = []

In [40]:
ref_all_seq = df_ref_transcript_data['Sequences'].tolist()

In [41]:
%%time
def get_gc_percentage():
    for i in tqdm(range(len(df_ref_transcript_data['Sequences']))):
        
        if type(df_ref_transcript_data['Sequences'][i]) is float:
            gc_percentage.append('Nan')
            
        else: 
            gc_percentage.append(GC123(df_ref_transcript_data['Sequences'][i])[0])
            

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.87 µs


In [42]:
%%time
get_gc_percentage()


 47%|████▋     | 12087/25715 [00:35<00:38, 352.03it/s]


 97%|█████████▋| 25031/25715 [01:09<00:02, 257.54it/s]


100%|██████████| 25715/25715 [01:10<00:00, 363.18it/s]

CPU times: user 1min 5s, sys: 927 ms, total: 1min 6s
Wall time: 1min 10s


In [43]:
c = 0
index = []
for i in tqdm(range(len(gc_percentage))):
    
    if gc_percentage[i] == 'Nan':
        c = c + 1
        index.append(i)
        continue
        
    else:
        gc_percentage[i] = float("{0:.2f}".format(gc_percentage[i]))


100%|██████████| 25715/25715 [00:00<00:00, 568944.56it/s]

In [44]:
index

[25622, 25623, 25641, 25646, 25647, 25693, 25714]

In [45]:
c

7

In [47]:
gc_percentage[0:5]

[51.3, 51.74, 44.27, 39.8, 42.8]

In [48]:
a = list(df_ref_transcript_data.columns.values)

In [53]:
a

['SNO',
 'Index',
 'RefSeq Gene ID',
 'RefSeq Transcript ID',
 'Dbxref',
 'Parent',
 'gbkey',
 'Type',
 'Seq_id',
 'Strand',
 'Start',
 'End',
 'No. of Exons',
 'Transcript Length',
 'Length of Sequences',
 '| Seq_len - Trans_len |',
 'Match ?',
 'Sequences',
 'Len of longest ORF',
 'No. of Amino Acids',
 'Longest ORF Seq',
 'ORF Start Index for longest Seq',
 'Protein Seq encoded',
 'Stop codon present ?',
 'ORF multiple of 3 ?']

In [50]:
b = a[0:18]

b.append('GC %')

df_ref_transcript_data['GC %'] = gc_percentage

c = b + a[18:]

# c

df_ref_transcript_data = df_ref_transcript_data[c]

In [51]:
df_ref_transcript_data.head(3)

,SNO,Index,RefSeq Gene ID,RefSeq Transcript ID,Dbxref,Parent,gbkey,Type,Seq_id,Strand,...,Match ?,Sequences,GC %,Len of longest ORF,No. of Amino Acids,Longest ORF Seq,ORF Start Index for longest Seq,Protein Seq encoded,Stop codon present ?,ORF multiple of 3 ?
0,1,27,gene-MIR1302-2HG,rna-XR_001737835.1,"GeneID:107985730,Genbank:XR_001737835.1,HGNC:H...",gene-MIR1302-2HG,ncRNA,lnc_RNA,1,1,...,YES,TCGTATGAGTCCAGTTTTGTGAAGTGCCTAGAATAGTCAAAATTCA...,51.3,15,5,ATGAGTCCAGTTTTGTGA,4,MSPVL,YES,YES
1,1508,251564,gene-LOC107985451,rna-XR_001737807.1,"GeneID:107985451,Genbank:XR_001737807.1",gene-LOC107985451,ncRNA,lnc_RNA,1,-1,...,YES,GCCATTAAGGGATGAGCGTAAGCAGTGACTGACAGGCAGGCTCTGC...,51.74,18,6,ATGAGCGTAAGCAGTGACTGA,11,MSVSSD,YES,YES
2,1507,251556,gene-LOC100505795,rna-NR_038072.1,"GeneID:100505795,Genbank:NR_038072.1",gene-LOC100505795,ncRNA,lnc_RNA,1,-1,...,YES,CTTATTAAAAATAGAATATTGCCTACATCACATAAAGGGCAAACTC...,44.27,123,41,ATGGGCACACTTGGTGATGTGCAACTCAGTGGGGCCTACAGGGTCC...,118,MGTLGDVQLSGAYRVPADIDITKEFEPGLRNRVSGCFKKAV,YES,YES


In [55]:
len(df_ref_transcript_data.iloc[0])

26

In [56]:
len(c)

26

In [57]:
df_ref_transcript_data.to_csv('/Users/pankajverma/Desktop/summerInternship2k17/GSoC/Ensembl/1.lncRNA/feature_selection/ORF/RefSeq/all_lncrna_transcript_data_orf_info.csv')